In [ ]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
import sys
import os
import plotly.express as px
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

In [ ]:
roadmapSelected = 'A'

In [ ]:
UI_params = {
    'initial_procurement_budget':350000,
    'budget_rate':0.03,
    'initial_operations_budget':300000,
    'planning_interval':[2022,2037],
    'emissions_baseline': data.emissions2020.sum(),#metric tons
    'emissions_target_pct_of_base':0.40,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'objective_weights':{'cost':1,'emissions':0},
}

In [ ]:
data = pd.read_excel('../data/04APR_data_template.xlsx').head(100)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [ ]:
%%time
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)

In [ ]:
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [ ]:
replacement_details = {}

for v in inputs.vehicle_idx:
    for s in validSchedulesPerVehicle[v]:
        decision = x[v,s].x
        if decision == 1:
            sched = inputs.replacement_schedules[v,s,:]
            
            try:
                year_replaced = np.where(sched==1)[0][0]
                prior_year = year_replaced - 1
            except:
                year_replaced = 999
                prior_year = 999
            
            #special case: prior year = -1, this means it was replaced right away
            if prior_year == -1:
                
                age_replaced = inputs.current_vehicle_ages[v]
                vmt_replaced = inputs.data.miles2020[v]
                odometer_replaced = inputs.data.cumulative_miles[v]
            
            #special case 2: prior year = 998, this means it was never replaced (ignoring for now)
            elif prior_year == 999:
                age_replaced = 999
                vmt_replaced = 999
            
            else:
                age_replaced = inputs.age[v,s,prior_year]
                vmt_replaced = inputs.annual_mileage[v,s,prior_year]
                odometer_replaced = inputs.odometer[v,s,prior_year]
                
            replacement_details[v] = (year_replaced,age_replaced,vmt_replaced,odometer_replaced)

In [ ]:
results = pd.DataFrame(replacement_details).T
results.reset_index(inplace=True)
results.columns = ['vehicle_idx','year_replaced','age_replaced','annual_mileage_replaced','odometer_replaced']

In [ ]:
results = results.merge(data[['vehicle_idx','replacement_vehicle_purchaseprice']],on='vehicle_idx',how='left')

In [ ]:
resultsReplaced = results.copy()

In [ ]:
resultsReplaced = resultsReplaced[resultsReplaced.year_replaced!=999]

In [ ]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

In [ ]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

In [ ]:
np.median(inputs.annual_mileage)*.18*1458

In [ ]:
visualize_aquisition_cost(roadmapSelected,options,inputs)

In [ ]:
visualize_operations_cost(roadmapSelected,options,inputs)

In [ ]:
#i want to know what year each vehicle was replaced (don't necessarily need first)
#then what was the age of the vehicle the year before that
#what was the odometer the year before that
#what was the usage the year before that

#you can plot replacement year against all of these, as well as acquisition cost, mx, etc.




In [ ]:
fig = px.scatter(resultsReplaced, x="year_replaced", y="annual_mileage_replaced",color_discrete_sequence=["orange"])#,color="odometer_replaced")#,size="odometer_replaced")
fig.update_layout(yaxis_range=[0,65000],title='Year Vehicle Replaced in Model vs Annual Mileage')
fig.show()